In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary

import json

In [11]:
# dataset

boston_path = '../json_data/boston.json'
with open(boston_path, 'r') as f:
    boston_json = json.load(f)
    
print(type(boston_json))
for key in boston_json.keys():
    print(key, ': ', len(boston_json[key]))

<class 'dict'>
CRIM :  506
ZN :  506
INDUS :  506
CHAS :  506
NOX :  506
RM :  506
AGE :  506
DIS :  506
RAD :  506
TAX :  506
PTRATIO :  506
B :  506
LSTAT :  506
MEDV :  506


In [12]:
boston_df = pd.DataFrame.from_dict(boston_json)
print(boston_df.head())

      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31     0  0.538  6.575  65.2  4.0900    1  296.0   
1  0.02731   0.0   7.07     0  0.469  6.421  78.9  4.9671    2  242.0   
2  0.02729   0.0   7.07     0  0.469  7.185  61.1  4.9671    2  242.0   
3  0.03237   0.0   2.18     0  0.458  6.998  45.8  6.0622    3  222.0   
4  0.06905   0.0   2.18     0  0.458  7.147  54.2  6.0622    3  222.0   

   PTRATIO       B  LSTAT  MEDV  
0     15.3  396.90   4.98  24.0  
1     17.8  396.90   9.14  21.6  
2     17.8  392.83   4.03  34.7  
3     18.7  394.63   2.94  33.4  
4     18.7  396.90   5.33  36.2  


In [36]:
boston_np = np.array(boston_df)
print("dataset shape: ", boston_np.shape)

data_x = boston_np[:, :-1]
data_y = boston_np[:, -1].reshape(-1, 1)

x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size = 0.05)
print("x shape: ", x_train.shape, ", ", x_test.shape)
print("y shape: ", y_train.shape, ", ", y_test.shape)

dataset shape:  (506, 14)
x shape:  (480, 13) ,  (26, 13)
y shape:  (480, 1) ,  (26, 1)


In [39]:
# norm

x_scaler = StandardScaler()
y_scaler = StandardScaler()
x_scaler.fit(x_train)
y_scaler.fit(y_train)

x_train = x_scaler.transform(x_train)
y_train = y_scaler.transform(y_train)
x_test = x_scaler.transform(x_test)
y_test = y_scaler.transform(y_test)

In [40]:
# dataset

class dataset(Dataset):
    
    def __init__ (self, x, y):
        self.x = x
        self.y = y
        self.data = list(zip(self.x, self.y))

    def __getitem__(self, index):
        assert index < len(self.data)
        return self.data[index]

    def __len__(self):
        return len(self.data)

In [44]:
class boston_model(nn.Module):
    
    def __init__(self, in_layer, out_layer):
        super().__init__()
        self.linear = nn.Sequential(
            nn.Linear(in_layer, 50),
            nn.ReLU(),
            nn.Linear(50, 50),
            nn.ReLU(),
            nn.Linear(50, out_layer)
            )
        
    def forward(self, x):
        return self.linear(x)
    
model = boston_model(14, 1)
print(model)

boston_model(
  (linear): Sequential(
    (0): Linear(in_features=14, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=1, bias=True)
  )
)


In [ ]:
# # helper function for classification

# def calc_acc(out, labels):
#     num = out.size(0)
#     prediction = out.argmax(dim=1)
#     return (prediction == labels).sum().item()/num

# def evaluate(model, loss_func, dataloader):
#     loss = 0
#     acc = 0

#     loss = []
#     acc = []
#     for step, (inputs, labels) in enumerate(dataloader):
#         inputs = inputs.to(device)
#         labels = labels.to(device)

#         out = model(inputs)
#         loss.append(loss_func(out, labels))
#         acc.append(calc_acc(out, labels))

#     loss = torch.mean(torch.tensor(loss))
#     acc = torch.mean(torch.tensor(acc))
#     return loss, acc

In [ ]:
# helper function for regression

def calc_acc(y_hat, y):
    